In [2]:
import time
import selenium
import requests
import re
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from requests.exceptions import HTTPError
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

In [2]:
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)


In [3]:
driver.get('https://www.olx.com.br/imoveis/aluguel/casas/estado-mg/regiao-de-uberlandia-e-uberaba/uberlandia')

In [4]:
time.sleep(2)

In [5]:
driver.find_element(By.XPATH, '//*[@id="main-content"]/div[3]/div[1]/div[2]/button[2]').click()

In [6]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [7]:
all_houses = soup.find_all('section', attrs={'class': 'olx-ad-card olx-ad-card--vertical'})
house_list = list()


In [9]:
for house in all_houses:
    house_prices = house.find('h3', attrs={'class': re.compile(r'\bolx-ad-card__price\b')})
    if house_prices:
        house_prices = house_prices.text
        house_prices = house_prices.replace('R$', ' ').strip()
    else:
        house_prices = np.nan

    square_meters = house.find('span', attrs={'aria-label':re.compile(r'quadrado?')})
    garage = house.find('span', attrs={'aria-label':re.compile(r'garagem')})
    bathroom = house.find('span', attrs={'aria-label':re.compile(r'banheiro?')})
    bedroom = house.find('span', attrs={'aria-label':re.compile(r'quarto?')})
    address = house.find('p', attrs={'class':'olx-text olx-text--caption olx-text--block olx-text--regular'})
    
    if square_meters:
        square_meters = square_meters.text
        square_meters = square_meters.replace('m²', ' ').strip()
    else:
        square_meters = np.nan

    if garage:
        garage = garage.text
        garage = garage.replace('+', ' ').strip()
    else:
        garage = 0

    if bathroom:
        bathroom = bathroom.text
        bathroom = bathroom.replace('+', ' ').strip()
    else:
        bathroom = np.nan

    if bedroom:
        bedroom = bedroom.text
        bedroom = bedroom.replace('+', ' ').strip()
    else:
        bedroom = np.nan
    
    if address:
        address = address.text
        address = address.split(',')
        try:
            city = address[0].strip()
            neighborhood = address[1].strip()
        except IndexError:
            continue
            
        
    else:
        city = np.nan
        neighborhood = np.nan

    house_list.append([house_prices,bedroom, square_meters, garage, bathroom, city, neighborhood])


In [11]:
for i in house_list:
    print(i)

['1.500', '2', '80', '1', '3', 'Uberlândia', 'Centro']
['1.800', '3', '123', '4', '1', 'Uberlândia', 'Jardim Brasília']
['2.200', '3', '150', '2', '5', 'Uberlândia', 'Lagoinha']
['2.000', '3', '90', '1', '2', 'Uberlândia', 'Brasil']
['1.450', '2', '50', '1', '1', 'Uberlândia', 'Martins']
['3.800', '4', '244', '3', '4', 'Uberlândia', 'Santa Mônica']
['2.800', '2', '105', '2', '2', 'Uberlândia', 'Pacaembu']
['3.200', '3', '190', '5', '3', 'Uberlândia', 'Tibery']
['5.500', '1', '250', '3', '4', 'Uberlândia', 'Lídice']
['800', '3', '80', 0, '2', 'Uberlândia', 'Presidente Roosevelt']
['1.300', '3', '80', '1', '1', 'Uberlândia', 'Nossa Senhora Aparecida']
['1.700', '2', '70', '2', '2', 'Uberlândia', 'Novo Mundo']
['3.000', '3', '157', '2', '2', 'Uberlândia', 'Nossa Senhora Aparecida']
['2.400', '3', '150', '4', '2', 'Uberlândia', 'Tibery']
['20.000', '4', '491', 0, '5', 'Uberlândia', 'Jardim Karaíba']
['3.600', '3', '180', '3', '2', 'Uberlândia', 'Mansour']
['4.000', '3', '240', '3', '2', 'U

In [12]:
columns_name = ['Price', 'Bedrooms', 'Square_meters', 'Garage', 'Bathrooms', 'City', 'Neighborhood']
df = pd.DataFrame(house_list, columns=columns_name)

In [13]:
df

,Price,Bedrooms,Square_meters,Garage,Bathrooms,City,Neighborhood
0,1.500,2,80,1,3,Uberlândia,Centro
1,1.800,3,123,4,1,Uberlândia,Jardim Brasília
2,2.200,3,150,2,5,Uberlândia,Lagoinha
3,2.000,3,90,1,2,Uberlândia,Brasil
4,1.450,2,50,1,1,Uberlândia,Martins
5,3.800,4,244,3,4,Uberlândia,Santa Mônica
6,2.800,2,105,2,2,Uberlândia,Pacaembu
7,3.200,3,190,5,3,Uberlândia,Tibery
8,5.500,1,250,3,4,Uberlândia,Lídice
9,800,3,80,0,2,Uberlândia,Presidente Roosevelt
